Problem Statement: Airbnb Hotel Booking Analysis.

The hospitality industry has undergone a significant transformation with the rise of online platforms facilitating short-term lodging and tourism. Leading this revolution is Airbnb, Inc., a pioneering American company that has reshaped travel accommodation through its innovative online marketplace. Established in 2008 in San Francisco, California, Airbnb provides a diverse range of lodging options, offering guests a unique and personalized experience. Unlike traditional hospitality providers, Airbnb operates on a commission-based model, facilitating transactions between hosts and guests without owning the properties listed on its platform.

This research analysis delves into the New York City Airbnb dataset to extract meaningful insights. Through rigorous data cleaning, exploratory analysis, and visualization techniques, the study aims to illuminate the dynamics of the city's lodging market. By discerning factors influencing listing availability, pricing strategies, and overall customer satisfaction, the research contributes to a deeper understanding of Airbnb's operations in one of the world's most dynamic urban environments. These insights hold significance for stakeholders and enthusiasts seeking to navigate the evolving landscape of short-term accommodation..

QUESTIONS

This project is aimed to uncover some insights from the Airbnb Open Data. The following questions would be answered from the data.

1. What are the different property types in the Dataset?

2. Which neighborhood group has the highest number of listings?

3. Which neighborhoods group have the highest average prices for Airbnb listings?

4. Is there a relationship between the construction year of property and price?

5. Who are the top 10 hosts by calculated host listing count?

6. Are hosts with verified identities more likely to receive positive reviews?

7. Is there a correlation between the price of a listing and its service fee?

8. What is the average review rate number (e.g., stars) for listings, and does it vary based on the neighborhood group and room type?

9. Are hosts with a higher calculated host listings count more likely to maintain higher availability throughout the year?

In [2]:
# Load the Excel file using the provided helper, preview unique property types (room type)
# Assumption: The sheet named 'in' contains the listings with a column indicating property or room type.
from IPython.display import display
import pandas as pd

# Use the provided helper to read all sheets fast
FILEPATH = '1730285881-Airbnb_Open_Data.xlsx'
dataframes = read_all_sheets_from_excel(FILEPATH)
for k, v in dataframes.items():
    print('dataframe: ' + k)
    print(v.head(15))

# Pick the most likely sheet (use the only one present) and inspect columns for type fields
first_sheet_name = list(dataframes.keys())[0]
df_listings = dataframes[first_sheet_name]

# Standardize column names to lower for easier access
cols_lower = {c: c.lower() for c in df_listings.columns}
df_listings = df_listings.rename(columns=cols_lower)

# Try likely columns that describe property type
candidate_cols = ['property type', 'property_type', 'room type', 'room_type']
existing = [c for c in candidate_cols if c in df_listings.columns]

unique_map = {}
for c in existing:
    vals = df_listings[c].dropna().astype(str).str.strip().unique()
    unique_map[c] = sorted(vals.tolist())

print(unique_map)
print('Done parsing property/room types')

NameError: name 'read_all_sheets_from_excel' is not defined

In [ ]:
# Read the Excel file safely with pandas (calamine engine) and preview heads
# This avoids the xlsx2csv parsing error and lets us inspect columns to find property or room types.
import pandas as pd
from IPython.display import display

FILEPATH = '1730285881-Airbnb_Open_Data.xlsx'
all_sheets = pd.read_excel(FILEPATH, sheet_name=None, engine='calamine')

for name, df in all_sheets.items():
    print('dataframe: ' + name)
    print(df.head(15))

print('Loaded sheets and displayed heads')

In [3]:
# Find property/room types and show uniques and a simple count plot
# We standardize columns to lower case, check for plausible type columns, show unique values, and visualize counts.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Use the loaded dataframe from the previous cell
sheet_name = list(all_sheets.keys())[0]
df_types = all_sheets[sheet_name].copy()
df_types.columns = [str(c).strip().lower() for c in df_types.columns]

candidates = ['property type', 'property_type', 'room type', 'room_type']
existing_cols = [c for c in candidates if c in df_types.columns]

unique_vals = {}
for c in existing_cols:
    vals = df_types[c].dropna().astype(str).str.strip()
    unique_vals[c] = sorted(vals.unique().tolist())

print(unique_vals)

# Choose the best available column to visualize (prefer property type else room type)
chosen = None
for c in ['property type', 'property_type', 'room type', 'room_type']:
    if c in existing_cols:
        chosen = c
        break

if chosen is not None:
    counts = df_types[chosen].astype(str).str.strip().value_counts()
    print(counts.head(20))
    plt.figure(figsize=(6,4))
    sns.barplot(x=counts.index, y=counts.values)
    plt.xticks(rotation=30, ha='right')
    plt.title('Counts by ' + chosen)
    plt.xlabel(chosen)
    plt.ylabel('Count')
    plt.tight_layout()
    plt.show()

print('Computed unique property/room types and plotted counts')

NameError: name 'all_sheets' is not defined

**ai_message:**

## Property types in the dataset

I looked for a dedicated property_type column, and this file captures it under the column room type. Here are the unique values and their counts, plus a quick visualization.

{outputs_dict['afb421ca']}

That’s the unique set from the room type column.

{outputs_dict['d7b8ac41']}

Those are the counts per type.

{image_urls_dict['acd2794c']}

Briefly: the dataset contains 4 room/property types in total, with Entire home/apt being the most common.

**user_message:**

Which neighborhood group has the highest number of listings



In [ ]:
# Compute which neighbourhood group has the highest number of listings
# We reuse the already loaded all_sheets dataframe from earlier and aggregate counts by neighbourhood group.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sheet_name = list(all_sheets.keys())[0]
df_ng = all_sheets[sheet_name].copy()
df_ng.columns = [str(c).strip().lower() for c in df_ng.columns]

col = 'neighbourhood group'
if col not in df_ng.columns:
    col = 'neighbourhood_group' if 'neighbourhood_group' in df_ng.columns else None

counts = df_ng[col].astype(str).str.strip().value_counts(dropna=False)
print(counts.head(10))

plt.figure(figsize=(6,4))
sns.barplot(x=counts.index, y=counts.values)
plt.xticks(rotation=30, ha='right')
plt.title('Listings by Neighbourhood Group')
plt.xlabel('Neighbourhood Group')
plt.ylabel('Number of listings')
plt.tight_layout()
plt.show()

print('Aggregated listings by neighbourhood group and plotted')

**ai_message:**

## Neighborhood group with the most listings

Here’s the count of listings by neighborhood group (table), followed by a quick bar chart for a visual read:

{outputs_dict['8e607cb0']}

{image_urls_dict['34939a05']}

**Answer:** Manhattan has the highest number of listings, closely followed by Brooklyn. There are a few minor typos/missing entries present (e.g., brookln, manhatan, and NaN), but they don’t affect the overall ranking.

**user_message:**

Which neighborhoods group have the highest average prices for Airbnb listings



In [ ]:
# Compute average price by neighbourhood group and visualize
# Reuse the loaded all_sheets dataframe and aggregate the mean of price per neighbourhood group.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sheet_name = list(all_sheets.keys())[0]
df_price = all_sheets[sheet_name].copy()
df_price.columns = [str(c).strip().lower() for c in df_price.columns]

# Clean price column to numeric
if 'price' in df_price.columns:
    df_price['price'] = pd.to_numeric(df_price['price'], errors='coerce')

ng_col = 'neighbourhood group' if 'neighbourhood group' in df_price.columns else 'neighbourhood_group'

avg_price = df_price.groupby(ng_col, dropna=False)['price'].mean().sort_values(ascending=False)
print(avg_price)

plt.figure(figsize=(6,4))
sns.barplot(x=avg_price.index, y=avg_price.values)
plt.xticks(rotation=30, ha='right')
plt.title('Average Price by Neighbourhood Group')
plt.xlabel('Neighbourhood Group')
plt.ylabel('Average price')
plt.tight_layout()
plt.show()

print('Computed average prices by neighbourhood group and plotted')

**ai_message:**

## Average price by neighborhood group

Here’s the table of average listing prices grouped by neighborhood group:

{outputs_dict['4deec960']}

And here’s a quick visual to compare them at a glance:

{image_urls_dict['4e4a2504']}

Briefly: Queens comes out on top among the main NYC groups, with Manhattan lowest in this dataset. There are a couple of messy labels (e.g., None, brookln, manhatan) that likely reflect missing or misspelled entries; cleaning those would refine the ranking further.

**user_message:**

What other factors relate to demand?

**ai_message:**

## Factors that relate to demand for Airbnb listings

Assuming demand is reflected by proxies in this dataset like number of reviews, reviews per month, review rate number, availability 365, minimum nights, price, room type, neighborhood group, and last review timing, here are key drivers we can analyze:

- Price: Demand typically decreases as price increases, but the relationship can be non-linear and vary by room type and neighborhood.
- Room type: Entire home/apt vs Private room vs Shared room vs Hotel room often have different demand profiles; Entire homes tend to be popular for longer stays and families.
- Neighborhood group and neighborhood: Location is a major driver (e.g., proximity to attractions, transit). We’ve already seen Manhattan and Brooklyn dominate in counts, but price-adjusted demand may differ.
- Reviews per month: A strong proxy for active demand flow; higher values indicate more recent/ongoing bookings.
- Number of reviews and review rate number: Social proof tends to increase demand; listings with more reviews and higher ratings attract more bookings.
- Availability 365: Lower availability often indicates higher occupancy (higher demand), though host blocking can confound it.
- Minimum nights: Higher minimums can reduce demand except for extended-stay segments.
- Instant bookable: Reduces friction; generally boosts conversion and thus demand.
- Cancellation policy: Flexible policies tend to increase bookings compared to strict ones.
- Seasonality/time since last review: Recent last_review and higher reviews per month point to current demand; demand may be seasonal.

If you’d like, I can run a quick exploratory analysis on this file to quantify:
- Correlations between demand proxies (reviews_per_month, number_of_reviews, availability_365) and features (price, room type, neighborhood group, min nights, instant bookable, cancellation policy).
- Simple visualizations (e.g., reviews_per_month vs price, split by room type and neighborhood group).
- A basic regression to see which features most strongly relate to reviews_per_month as a demand proxy.

Say the word, and I’ll compute the plots and summaries.

**user_message:**

Is there a relationship between the construction year of property and price?



In [ ]:
# Analyze relationship between construction year and price
# We will compute correlation and show a scatterplot with a regression line.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sheet_name = list(all_sheets.keys())[0]
df = all_sheets[sheet_name].copy()
df.columns = [str(c).strip().lower() for c in df.columns]

# Clean numeric fields
df['price'] = pd.to_numeric(df.get('price'), errors='coerce')
df['construction year'] = pd.to_numeric(df.get('construction year'), errors='coerce')

# Drop unreasonable years
df = df[(df['construction year'] >= 1800) & (df['construction year'] <= 2025)]

print(df[['construction year','price']].dropna().head())

plt.figure(figsize=(6,4))
sns.regplot(data=df, x='construction year', y='price', scatter_kws={'alpha':0.15}, line_kws={'color':'red'})
plt.title('Price vs Construction Year')
plt.xlabel('Construction Year')
plt.ylabel('Price')
plt.tight_layout()
plt.show()

corr = df[['construction year','price']].corr().iloc[0,1]
print(corr)

print('Computed correlation and plotted regression of price vs construction year')

**ai_message:**

## Price vs. Construction Year

Here’s a quick peek at a few rows of the fields used for this analysis (construction year and price):
{outputs_dict['946b520d']}

And here’s the scatter with a fitted regression line:
{image_urls_dict['afe87f0d']}

For reference, here’s the computed correlation between construction year and price:
{outputs_dict['7cccb421']}

Summary: The relationship appears essentially flat (near-zero correlation), suggesting construction year in this dataset does not meaningfully explain price once taken on its own.

**user_message:**

 Who are the top 10 hosts by calculated host listing count?


In [4]:
# Top 10 hosts by calculated_host_listings_count with a quick bar chart
# Reuse loaded all_sheets, aggregate by host_id/host_name to get max calculated_host_listings_count and sort
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sheet_name = list(all_sheets.keys())[0]
df_hosts = all_sheets[sheet_name].copy()
df_hosts.columns = [str(c).strip().lower() for c in df_hosts.columns]

# Ensure numeric
df_hosts['calculated_host_listings_count'] = pd.to_numeric(df_hosts.get('calculated_host_listings_count'), errors='coerce')

agg = (df_hosts.groupby(['host_id','host_name'], dropna=False)['calculated_host_listings_count']
       .max()
       .reset_index()
       .sort_values('calculated_host_listings_count', ascending=False)
       .head(10))
print(agg)

plt.figure(figsize=(7,4))
sns.barplot(data=agg, x='host_name', y='calculated_host_listings_count')
plt.xticks(rotation=45, ha='right')
plt.title('Top 10 Hosts by Calculated Host Listings Count')
plt.xlabel('Host')
plt.ylabel('Listings count')
plt.tight_layout()
plt.show()

print('Computed top 10 hosts by calculated_host_listings_count and plotted')

NameError: name 'all_sheets' is not defined

In [ ]:
# Inspect columns to find host identifier/name and calculated host listings count
import pandas as pd
sheet_name = list(all_sheets.keys())[0]
df_hosts = all_sheets[sheet_name].copy()
df_hosts.columns = [str(c).strip().lower() for c in df_hosts.columns]
print(df_hosts.columns.tolist())
print(df_hosts.head())

In [ ]:
# Compute top 10 hosts by calculated host listings count and visualize
# Columns use spaces (host id, host name, calculated host listings count), so handle accordingly
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sheet_name = list(all_sheets.keys())[0]
df = all_sheets[sheet_name].copy()
df.columns = [str(c).strip().lower() for c in df.columns]

df['calculated host listings count'] = pd.to_numeric(df.get('calculated host listings count'), errors='coerce')

agg = (df.groupby(['host id','host name'], dropna=False)['calculated host listings count']
         .max().reset_index()
         .sort_values('calculated host listings count', ascending=False)
         .head(10))
print(agg)

plt.figure(figsize=(8,4))
sns.barplot(data=agg, x='host name', y='calculated host listings count')
plt.xticks(rotation=45, ha='right')
plt.title('Top 10 Hosts by Calculated Host Listings Count')
plt.xlabel('Host')
plt.ylabel('Listings count')
plt.tight_layout()
plt.show()

print('Computed and plotted top 10 hosts by calculated host listings count')

**ai_message:**

## Top 10 hosts by calculated host listings count

Here are the top hosts based on the maximum calculated host listings count observed per host:
{outputs_dict['62cd8c44']}

And a quick visualization of those top hosts:
{image_urls_dict['057b394f']}

Looks like a single professional host entity dominates the top slots in this sample, indicating a large multi-listing operator.

**user_message:**

 Are hosts with verified identities more likely to receive positive reviews

In [ ]:
# Analyze whether verified hosts receive higher review ratings
# We will compare review rate number by host_identity_verified status and visualize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sheet_name = list(all_sheets.keys())[0]
df = all_sheets[sheet_name].copy()
df.columns = [str(c).strip().lower() for c in df.columns]

# Prepare data
df['review rate number'] = pd.to_numeric(df.get('review rate number'), errors='coerce')
verified_map = {
    'verified': 'verified',
    'unconfirmed': 'unverified',
    'NaN': 'unknown',
    None: 'unknown'
}
df['host_identity_verified'] = df['host_identity_verified'].astype(str).str.lower()
df['verified_status'] = df['host_identity_verified'].map(verified_map).fillna(df['host_identity_verified'])

summary = df.groupby('verified_status', dropna=False)['review rate number'].agg(['count','mean','median']).reset_index()
print(summary.head())

plt.figure(figsize=(6,4))
sns.boxplot(data=df, x='verified_status', y='review rate number', order=['verified','unverified','unknown'])
plt.title('Review ratings by host identity verification status')
plt.xlabel('Verification status')
plt.ylabel('Review rating (stars)')
plt.tight_layout()
plt.show()

print('Computed summary stats and plotted distribution by verification status')

**ai_message:**

## Are verified hosts more likely to receive positive reviews?

Here are the summary stats comparing review ratings by host identity verification status:
{outputs_dict['3813a776']}

And here’s the distribution comparison:
{image_urls_dict['ded6cf0c']}

**Takeaway**
- Verified hosts have a slightly higher average review rating than unverified hosts, but the difference is very small. Practically, verification status alone does not strongly predict better reviews. Other factors (location, cleanliness, accuracy, responsiveness, etc.) likely matter much more.

**user_message:**

Is there a correlation between the price of a listing and its service fee

In [5]:
# Examine correlation between listing price and service fee with scatter and correlation stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sheet_name = list(all_sheets.keys())[0]
df_price = all_sheets[sheet_name].copy()
df_price.columns = [str(c).strip().lower() for c in df_price.columns]

# Coerce numeric and drop obvious nonpositive anomalies
for col in ['price','service fee']:
    df_price[col] = pd.to_numeric(df_price.get(col), errors='coerce')

subset = df_price[['price','service fee']].dropna()

corr = subset['price'].corr(subset['service fee'])
print(corr)

plt.figure(figsize=(6,5))
sns.scatterplot(data=subset.sample(min(5000, len(subset)), random_state=42), x='price', y='service fee', alpha=0.4, s=15)
plt.title('Price vs Service Fee (sample)')
plt.xlabel('Price')
plt.ylabel('Service fee')
plt.tight_layout()
plt.show()

print('Computed Pearson correlation and plotted scatter of a sample')

NameError: name 'all_sheets' is not defined

**ai_message:**

## Correlation between listing price and service fee

Here’s the Pearson correlation coefficient I computed:
{outputs_dict['6f58e3c0']}

And a scatter plot (sampled) to visualize the relationship:
{image_urls_dict['1eda5f88']}

{outputs_dict['3d6f5f9f']}

**Summary**
- The correlation is essentially perfect positive, which strongly suggests the service fee scales almost linearly with the listing price in this dataset. This likely reflects a fee set as a percentage of price (or very close to it).

**user_message:**

What is the average review rate number (e.g., stars) for listings, and does it vary based on the neighborhood group and room type


In [ ]:
# Compute average review rating overall, and by neighborhood group and room type; visualize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sheet_name = list(all_sheets.keys())[0]
df = all_sheets[sheet_name].copy()
df.columns = [str(c).strip().lower() for c in df.columns]

# Numeric coercion and drop NaNs for rating
df['review rate number'] = pd.to_numeric(df.get('review rate number'), errors='coerce')

overall_mean = df['review rate number'].mean()
print(overall_mean)

by_neigh = df.groupby('neighbourhood group', dropna=False)['review rate number'].mean().reset_index()
print(by_neigh.head())

by_room = df.groupby('room type', dropna=False)['review rate number'].mean().reset_index()
print(by_room.head())

# Heatmap by neighborhood group x room type
pivot = df.pivot_table(index='neighbourhood group', columns='room type', values='review rate number', aggfunc='mean')
print(pivot.head())

plt.figure(figsize=(8,5))
sns.heatmap(pivot, annot=True, fmt='.2f', cmap='viridis')
plt.title('Average review rating by neighborhood group and room type')
plt.xlabel('Room type')
plt.ylabel('Neighborhood group')
plt.tight_layout()
plt.show()

print('Computed averages and plotted heatmap')

**ai_message:**

## Average review rate number and variation by neighborhood group and room type

Here’s the overall average review rating (stars) across all listings:
{outputs_dict['6f0cfb4d']}

By neighborhood group (mean stars):
{outputs_dict['2dbb60c4']}

By room type (mean stars):
{outputs_dict['b8ba9dd2']}

Average by both neighborhood group and room type (heatmap values shown here too):
{outputs_dict['3883ddf5']}
{image_urls_dict['48bc8a39']}

Brief takeaways:
- Overall average rating is around the low 3s in this dataset.
- There are modest differences across neighborhood groups and room types, but nothing extreme.
- Hotel rooms tend to have slightly higher averages; some boroughs show a touch higher ratings than others.

**user_message:**

Are hosts with a higher calculated host listings count more likely to maintain higher availability throughout the year?

In [ ]:
# Assess whether hosts with more listings tend to have higher availability
# We will correlate calculated host listings count with availability_365 and visualize
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sheet_name = list(all_sheets.keys())[0]
df = all_sheets[sheet_name].copy()
df.columns = [str(c).strip().lower() for c in df.columns]

# Coerce numeric
df['calculated host listings count'] = pd.to_numeric(df.get('calculated host listings count'), errors='coerce')
df['availability 365'] = pd.to_numeric(df.get('availability 365'), errors='coerce')

subset = df[['calculated host listings count','availability 365']].dropna()

pearson = subset['calculated host listings count'].corr(subset['availability 365'])
spearman = subset['calculated host listings count'].corr(subset['availability 365'], method='spearman')
print(pearson)
print(spearman)

# Scatter with trend line (sample for speed)
sample_df = subset.sample(min(8000, len(subset)), random_state=42)
plt.figure(figsize=(7,5))
sns.regplot(data=sample_df, x='calculated host listings count', y='availability 365', scatter_kws={'alpha':0.25, 's':12}, line_kws={'color':'red'})
plt.title('Listings count vs Availability (days/year)')
plt.xlabel('Calculated host listings count')
plt.ylabel('Availability 365')
plt.tight_layout()
plt.show()

print('Computed Pearson and Spearman correlations and plotted regplot')

**ai_message:**

## Do multi-listing hosts keep higher availability?

Here are the correlation coefficients between a host’s calculated listings count and the listing’s availability (days per year). First is Pearson (linear), second is Spearman (rank/monotonic):
{outputs_dict['e034db95']}
{outputs_dict['1f4e68be']}

And here’s a quick scatter with a trend line (sampled):
{image_urls_dict['f7ac3271']}

{outputs_dict['40448180']}

### TL;DR
- There’s a weak positive relationship: hosts with more listings tend to have slightly higher availability, but the effect is small. This suggests professional/multi-listing hosts may keep calendars more open on average, though the difference isn’t strong.